<a href="https://colab.research.google.com/github/Aditya-Sarna/Aditya-Sarna-.github.io/blob/main/skincancerfinal_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import files
import os


!mkdir -p ~/.kaggle


files.upload()

!mv kaggle.json ~/.kaggle/


!chmod 600 ~/.kaggle/kaggle.json



Saving kaggle.json to kaggle.json
Kaggle API key setup complete. You can now try re-running the cell to download the dataset.


In [ ]:
import os
kaggle_config_path = os.path.expanduser("~/.kaggle/kaggle.json")
if os.path.exists(kaggle_config_path):
    print(f"Kaggle API key (kaggle.json) found at {kaggle_config_path}.")
else:
    print(f" Kaggle API key (kaggle.json) NOT found at {kaggle_config_path}. Please follow the setup instructions above.")

Kaggle API key (kaggle.json) found at /root/.kaggle/kaggle.json.


In [ ]:
import kagglehub
import os

path = kagglehub.dataset_download("kmader/skin-cancer-mnist-ham10000")
print("Dataset path:", path)
print(os.listdir(path))


100%|██████████| 5.20G/5.20G [00:56<00:00, 99.0MB/s]

Extracting files...


Dataset path: /root/.cache/kagglehub/datasets/kmader/skin-cancer-mnist-ham10000/versions/2
['HAM10000_metadata.csv', 'hmnist_28_28_L.csv', 'HAM10000_images_part_1', 'HAM10000_images_part_2', 'hmnist_8_8_L.csv', 'ham10000_images_part_1', 'hmnist_8_8_RGB.csv', 'ham10000_images_part_2', 'hmnist_28_28_RGB.csv']


In [ ]:
DATA_CSV = "/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv"

IMAGE_DIR_1 = "/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_1"
IMAGE_DIR_2 = "/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_2"



In [ ]:
IMAGE_DIRS = [
    "/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_1",
    "/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_2"
]


In [ ]:
def find_image_path(image_id, image_dirs):
    for d in image_dirs:
        img_path = os.path.join(d, image_id + ".jpg")
        if os.path.exists(img_path):
            return Image.open(img_path)
    return None

In [ ]:
import os, shutil

DEST = "/content/HAM10000/images"
os.makedirs(DEST, exist_ok=True)

src1 = "/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_1"
src2 = "/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_2"

for f in os.listdir(src1):
    shutil.copy(os.path.join(src1, f), DEST)

for f in os.listdir(src2):
    shutil.copy(os.path.join(src2, f), DEST)

print("Total images:", len(os.listdir(DEST)))


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_1'

In [ ]:
!pip install timm --quiet
!pip install scikit-learn --quiet


import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import timm
from tqdm import tqdm


DATA_CSV = "/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv"
IMAGE_DIRS = [
    "/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_1",
    "/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_2"
]


df = pd.read_csv(DATA_CSV)


label_map = {label: i for i, label in enumerate(df["dx"].unique())}
df["label"] = df["dx"].map(label_map)

print("Label mapping:", label_map)
print(df.head())

class HAMDataset(Dataset):
    def __init__(self, df, image_dirs, transform=None):
        self.df = df
        self.image_dirs = image_dirs
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_id = row["image_id"] + ".jpg"


        img_path = None
        for d in self.image_dirs:
            path = os.path.join(d, image_id)
            if os.path.exists(path):
                img_path = path
                break

        if img_path is None:
            raise FileNotFoundError(f"Image not found: {image_id}")

        img = Image.open(img_path).convert("RGB")
        label = row["label"]

        if self.transform:
            img = self.transform(img)

        return img, label


IMG_SIZE = 224

train_tf = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

val_tf = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])


train_df, val_df = train_test_split(
    df, test_size=0.15, stratify=df["label"], random_state=42
)

train_ds = HAMDataset(train_df, IMAGE_DIRS, train_tf)
val_ds = HAMDataset(val_df, IMAGE_DIRS, val_tf)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=2)

print("Train images:", len(train_ds))
print("Val images:", len(val_ds))


device = "cuda" if torch.cuda.is_available() else "cpu"

model = timm.create_model("vit_base_patch16_224", pretrained=True)
model.head = nn.Linear(model.head.in_features, 7)   # 7 classes
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)

scaler = torch.amp.GradScaler("cuda")


best_acc = 0
patience = 5
wait = 0

for epoch in range(30):


    model.train()
    correct = 0
    total = 0
    running_loss = 0

    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/30"):
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()

        with torch.amp.autocast("cuda"):
            outputs = model(imgs)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item() * imgs.size(0)
        _, pred = outputs.max(1)
        correct += pred.eq(labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total


    model.eval()
    val_correct = 0
    val_total = 0

    with torch.no_grad(), torch.amp.autocast("cuda"):
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, pred = outputs.max(1)
            val_correct += pred.eq(labels).sum().item()
            val_total += labels.size(0)

    val_acc = val_correct / val_total

    print(f"Epoch {epoch+1}: Train Acc {train_acc:.4f} | Val Acc {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_vit_ham10000.pth")
        wait = 0
        print("✔ Saved new best model")
    else:
        wait += 1
        if wait >= patience:
            print(" Early Stopping")
            break


model.load_state_dict(torch.load("best_vit_ham10000.pth"))
model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for imgs, labels in val_loader:
        imgs = imgs.to(device)
        outputs = model(imgs)
        preds = outputs.argmax(1).cpu().numpy()

        all_preds.extend(preds)
        all_labels.extend(labels.numpy())

print("\n=== CLASSIFICATION REPORT ===")
print(classification_report(all_labels, all_preds, target_names=df["dx"].unique()))

print("Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv'

In [ ]:
from PIL import Image
import torch
import torch.nn as nn
from torchvision import transforms
import timm
import matplotlib.pyplot as plt
import numpy as np
from google.colab import files



idx_to_class = {
    0: "bkl",
    1: "nv",
    2: "df",
    3: "mel",
    4: "vasc",
    5: "bcc",
    6: "akiec"
}

cancer_classes = {"mel", "bcc", "akiec"}


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])



model_path = "best_vit_ham10000.pth"

model = timm.create_model("vit_base_patch16_224", pretrained=False)
model.head = nn.Linear(model.head.in_features, 7)
model.load_state_dict(torch.load(model_path, map_location="cpu"))
model.eval()



uploaded = files.upload()
img_path = list(uploaded.keys())[0]

img = Image.open(img_path).convert("RGB")
img_tensor = transform(img).unsqueeze(0)



with torch.no_grad():
    output = model(img_tensor)
    pred = torch.argmax(output, 1).item()

pred_class = idx_to_class[pred]
cancer_status = "CANCER" if pred_class in cancer_classes else "NOT CANCER"



plt.imshow(img)
plt.axis("off")
plt.title(f"Prediction: {pred_class} | {cancer_status}", fontsize=16)
plt.show()

print("\n=== RESULT ===")
print("Predicted Class:", pred_class)
print("Cancer Status:", cancer_status)
